In [0]:
!pip install scikit-learn
!pip install numpy==1.16.1

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'


In [0]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

In [0]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [0]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2
/device:GPU:0
Num GPUs Available:  1


In [0]:
def Model(num_classes):
  layers = [
      tf.keras.layers.Dense(128, activation = tf.nn.relu),
      tf.keras.layers.Dense(64, activation = tf.nn.relu),
      tf.keras.layers.Dense(32, activation = tf.nn.relu),
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ]
  model = tf.keras.Sequential(layers)
  return model

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  train_idx = folds[0][0]
  test_idx = folds[0][1]
  X_train = X[train_idx]
  X_test = X[test_idx]
  Total_Ytrain = Y[train_idx]
  Total_Ytest = Y[test_idx]
  Total_Xtrain = feature_extraction(X_train)
  Total_Xtest = feature_extraction(X_test)

  for i in range(1, len(folds)):
          train_idx = folds[i][0]
          test_idx = folds[i][1]

          X_train = X[train_idx]
          X_test = X[test_idx]
          Y_train = Y[train_idx]
          Y_test = Y[test_idx]
          X_train = feature_extraction(X_train)
          X_test = feature_extraction(X_test)
          Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
          Total_Ytrain=np.append(Total_Ytrain, Y_train)
          Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
          Total_Ytest=np.append(Total_Ytest, Y_test)


  print(Total_Xtrain.shape, Total_Ytrain.shape)
  print(Total_Xtest.shape,Total_Ytest.shape)

  model = Model(num_classes)
  es = EarlyStopping(monitor='sparse_categorical_accuracy', mode='max', verbose=1, patience=15)
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(Total_Xtrain, Total_Ytrain, batch_size=32, epochs=200,callbacks=[es])
  _,acc=model.evaluate(Total_Xtest,Total_Ytest)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [0]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [0]:
Run('data/LOSO/MHEALTH.npz')

Proposed Model data/LOSO/MHEALTH.npz
12
(22995, 93) (22995,)
(2555, 93) (2555,)
Epoch 1/100
719/719 [==============================] - 2s 3ms/step - loss: 0.2638 - sparse_categorical_accuracy: 0.9417
Epoch 2/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0209 - sparse_categorical_accuracy: 0.9927
Epoch 3/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0114 - sparse_categorical_accuracy: 0.9955
Epoch 4/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9994
Epoch 5/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0163 - sparse_categorical_accuracy: 0.9949
Epoch 6/100
719/719 [==============================] - 2s 3ms/step - loss: 3.5715e-04 - sparse_categorical_accuracy: 1.0000
Epoch 7/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0066 - sparse_categorical_accuracy: 0.9978
Epoch 8/100
719/719 [==============================] - 2s 3ms/step - loss: 9

In [0]:
Run('data/LOTO/MHEALTH.npz')

Proposed Model data/LOTO/MHEALTH.npz
12
(22995, 93) (22995,)
(2555, 93) (2555,)
Epoch 1/100
719/719 [==============================] - 2s 3ms/step - loss: 0.2547 - sparse_categorical_accuracy: 0.9452
Epoch 2/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0225 - sparse_categorical_accuracy: 0.9927
Epoch 3/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0169 - sparse_categorical_accuracy: 0.9940
Epoch 4/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0034 - sparse_categorical_accuracy: 0.9990
Epoch 5/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0377 - sparse_categorical_accuracy: 0.9899
Epoch 6/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0081 - sparse_categorical_accuracy: 0.9973
Epoch 7/100
719/719 [==============================] - 2s 3ms/step - loss: 3.5069e-04 - sparse_categorical_accuracy: 1.0000
Epoch 8/100
719/719 [==============================] - 2s 3ms/step - loss: 0

In [0]:
Run('data/FNOW/MHEALTH.npz')

Proposed Model data/FNOW/MHEALTH.npz
12
(12015, 93) (12015,)
(1335, 93) (1335,)
Epoch 1/100
376/376 [==============================] - 1s 3ms/step - loss: 0.3589 - sparse_categorical_accuracy: 0.9134
Epoch 2/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0440 - sparse_categorical_accuracy: 0.9858
Epoch 3/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0217 - sparse_categorical_accuracy: 0.9928
Epoch 4/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0086 - sparse_categorical_accuracy: 0.9967
Epoch 5/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0030 - sparse_categorical_accuracy: 0.9993
Epoch 6/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0395 - sparse_categorical_accuracy: 0.9903
Epoch 7/100
376/376 [==============================] - 1s 3ms/step - loss: 0.0019 - sparse_categorical_accuracy: 0.9996
Epoch 8/100
376/376 [==============================] - 1s 3ms/step - loss: 4.073

In [0]:
Run('data/SNOW/MHEALTH.npz')

Proposed Model data/SNOW/MHEALTH.npz
12
(22995, 93) (22995,)
(2555, 93) (2555,)
Epoch 1/100
719/719 [==============================] - 2s 3ms/step - loss: 0.3815 - sparse_categorical_accuracy: 0.9329
Epoch 2/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0210 - sparse_categorical_accuracy: 0.9927
Epoch 3/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0104 - sparse_categorical_accuracy: 0.9966
Epoch 4/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0127 - sparse_categorical_accuracy: 0.9956
Epoch 5/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0104 - sparse_categorical_accuracy: 0.9968
Epoch 6/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0012 - sparse_categorical_accuracy: 0.9995
Epoch 7/100
719/719 [==============================] - 2s 3ms/step - loss: 0.0222 - sparse_categorical_accuracy: 0.9933
Epoch 8/100
719/719 [==============================] - 2s 3ms/step - loss: 5.682

In [0]:
Run('data/LOTO/USCHAD.npz')

Proposed Model data/LOTO/USCHAD.npz
12
(88416, 25) (88416,)
(9824, 25) (9824,)
Epoch 1/100
2763/2763 [==============================] - 7s 2ms/step - loss: 0.4757 - sparse_categorical_accuracy: 0.8167
Epoch 2/100
2763/2763 [==============================] - 7s 2ms/step - loss: 0.2407 - sparse_categorical_accuracy: 0.8956
Epoch 3/100
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1959 - sparse_categorical_accuracy: 0.9114
Epoch 4/100
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1682 - sparse_categorical_accuracy: 0.9225
Epoch 5/100
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1528 - sparse_categorical_accuracy: 0.9273
Epoch 6/100
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1357 - sparse_categorical_accuracy: 0.9354
Epoch 7/100
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1279 - sparse_categorical_accuracy: 0.9389
Epoch 8/100
2763/2763 [==============================] - 7s 2ms/ste

In [0]:
Run('data/LOSO/USCHAD.npz')

Proposed Model data/LOSO/USCHAD.npz
12
(127712, 25) (127712,)
(9824, 25) (9824,)
Epoch 1/100
3991/3991 [==============================] - 10s 2ms/step - loss: 0.4530 - sparse_categorical_accuracy: 0.8287
Epoch 2/100
3991/3991 [==============================] - 10s 2ms/step - loss: 0.2119 - sparse_categorical_accuracy: 0.9064
Epoch 3/100
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1675 - sparse_categorical_accuracy: 0.9221
Epoch 4/100
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1462 - sparse_categorical_accuracy: 0.9323
Epoch 5/100
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1277 - sparse_categorical_accuracy: 0.9398
Epoch 6/100
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1192 - sparse_categorical_accuracy: 0.9446
Epoch 7/100
3991/3991 [==============================] - 10s 2ms/step - loss: 0.1089 - sparse_categorical_accuracy: 0.9483
Epoch 8/100
3991/3991 [==============================] - 1

In [0]:
Run('data/FNOW/USCHAD.npz')

Proposed Model data/FNOW/USCHAD.npz
12
(45342, 25) (45342,)
(5038, 25) (5038,)
Epoch 1/100
1417/1417 [==============================] - 4s 3ms/step - loss: 0.6349 - sparse_categorical_accuracy: 0.7691
Epoch 2/100
1417/1417 [==============================] - 4s 3ms/step - loss: 0.3040 - sparse_categorical_accuracy: 0.8750
Epoch 3/100
1417/1417 [==============================] - 4s 3ms/step - loss: 0.2321 - sparse_categorical_accuracy: 0.9000
Epoch 4/100
1417/1417 [==============================] - 4s 3ms/step - loss: 0.2003 - sparse_categorical_accuracy: 0.9131
Epoch 5/100
1417/1417 [==============================] - 4s 3ms/step - loss: 0.1731 - sparse_categorical_accuracy: 0.9216
Epoch 6/100
1417/1417 [==============================] - 4s 3ms/step - loss: 0.1596 - sparse_categorical_accuracy: 0.9274
Epoch 7/100
1417/1417 [==============================] - 4s 3ms/step - loss: 0.1446 - sparse_categorical_accuracy: 0.9333
Epoch 8/100
1417/1417 [==============================] - 4s 3ms/ste

In [0]:
Run('data/SNOW/USCHAD.npz')

Proposed Model data/SNOW/USCHAD.npz
12
(88416, 25) (88416,)
(9824, 25) (9824,)
Epoch 1/200
2763/2763 [==============================] - 7s 3ms/step - loss: 0.4899 - sparse_categorical_accuracy: 0.8120
Epoch 2/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.2414 - sparse_categorical_accuracy: 0.8947
Epoch 3/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1946 - sparse_categorical_accuracy: 0.9135
Epoch 4/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1628 - sparse_categorical_accuracy: 0.9236
Epoch 5/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1495 - sparse_categorical_accuracy: 0.9297
Epoch 6/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1402 - sparse_categorical_accuracy: 0.9347
Epoch 7/200
2763/2763 [==============================] - 7s 2ms/step - loss: 0.1292 - sparse_categorical_accuracy: 0.9378
Epoch 8/200
2763/2763 [==============================] - 7s 2ms/ste

In [0]:
Run('data/LOTO/WHARF.npz')

Proposed Model data/LOTO/WHARF.npz
12
(34839, 13) (34839,)
(3871, 13) (3871,)
Epoch 1/200
1089/1089 [==============================] - 3s 3ms/step - loss: 1.2678 - sparse_categorical_accuracy: 0.5868
Epoch 2/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.9630 - sparse_categorical_accuracy: 0.6527
Epoch 3/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.8757 - sparse_categorical_accuracy: 0.6719
Epoch 4/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.8223 - sparse_categorical_accuracy: 0.6886
Epoch 5/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.7796 - sparse_categorical_accuracy: 0.7020
Epoch 6/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.7471 - sparse_categorical_accuracy: 0.7138
Epoch 7/200
1089/1089 [==============================] - 3s 3ms/step - loss: 0.7189 - sparse_categorical_accuracy: 0.7199
Epoch 8/200
1089/1089 [==============================] - 3s 3ms/step

In [0]:
Run('data/LOSO/WHARF.npz')

Proposed Model data/LOSO/WHARF.npz
12
(58065, 13) (58065,)
(3871, 13) (3871,)
Epoch 1/200
1815/1815 [==============================] - 5s 3ms/step - loss: 1.0504 - sparse_categorical_accuracy: 0.6334
Epoch 2/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.8149 - sparse_categorical_accuracy: 0.6940
Epoch 3/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.7597 - sparse_categorical_accuracy: 0.7094
Epoch 4/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.7206 - sparse_categorical_accuracy: 0.7224
Epoch 5/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.6869 - sparse_categorical_accuracy: 0.7310
Epoch 6/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.6643 - sparse_categorical_accuracy: 0.7391
Epoch 7/200
1815/1815 [==============================] - 5s 3ms/step - loss: 0.6376 - sparse_categorical_accuracy: 0.7491
Epoch 8/200
1815/1815 [==============================] - 5s 3ms/step

In [0]:
Run('data/SNOW/WHARF.npz')

Proposed Model data/SNOW/WHARF.npz
12
(34920, 13) (34920,)
(3880, 13) (3880,)
Epoch 1/200
1092/1092 [==============================] - 3s 3ms/step - loss: 1.3384 - sparse_categorical_accuracy: 0.5698
Epoch 2/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.9399 - sparse_categorical_accuracy: 0.6530
Epoch 3/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.8453 - sparse_categorical_accuracy: 0.6811
Epoch 4/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.8083 - sparse_categorical_accuracy: 0.6919
Epoch 5/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.7728 - sparse_categorical_accuracy: 0.7059
Epoch 6/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.7485 - sparse_categorical_accuracy: 0.7141
Epoch 7/200
1092/1092 [==============================] - 3s 3ms/step - loss: 0.7258 - sparse_categorical_accuracy: 0.7219
Epoch 8/200
1092/1092 [==============================] - 3s 3ms/step

In [0]:
Run('data/FNOW/WHARF.npz')

Proposed Model data/FNOW/WHARF.npz
12
(19314, 13) (19314,)
(2146, 13) (2146,)
Epoch 1/200
604/604 [==============================] - 2s 3ms/step - loss: 1.4125 - sparse_categorical_accuracy: 0.5397
Epoch 2/200
604/604 [==============================] - 2s 3ms/step - loss: 1.0380 - sparse_categorical_accuracy: 0.6167
Epoch 3/200
604/604 [==============================] - 2s 3ms/step - loss: 0.9576 - sparse_categorical_accuracy: 0.6427
Epoch 4/200
604/604 [==============================] - 2s 3ms/step - loss: 0.9051 - sparse_categorical_accuracy: 0.6559
Epoch 5/200
604/604 [==============================] - 2s 3ms/step - loss: 0.8504 - sparse_categorical_accuracy: 0.6724
Epoch 6/200
604/604 [==============================] - 2s 3ms/step - loss: 0.8182 - sparse_categorical_accuracy: 0.6840
Epoch 7/200
604/604 [==============================] - 2s 3ms/step - loss: 0.7898 - sparse_categorical_accuracy: 0.6921
Epoch 8/200
604/604 [==============================] - 2s 3ms/step - loss: 0.7659 

In [0]:
Run('data/LOTO/WISDM.npz')

Proposed Model data/LOTO/WISDM.npz
6
(187614, 13) (187614,)
(20846, 13) (20846,)
Epoch 1/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.3325 - sparse_categorical_accuracy: 0.8804
Epoch 2/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.1884 - sparse_categorical_accuracy: 0.9316
Epoch 3/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.1406 - sparse_categorical_accuracy: 0.9501
Epoch 4/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.1167 - sparse_categorical_accuracy: 0.9592
Epoch 5/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.1023 - sparse_categorical_accuracy: 0.9642
Epoch 6/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.0918 - sparse_categorical_accuracy: 0.9678
Epoch 7/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.0834 - sparse_categorical_accuracy: 0.9710
Epoch 8/200
5863/5863 [==============================] - 1

In [0]:
Run('data/LOSO/WISDM.npz')

Proposed Model data/LOSO/WISDM.npz
6
(729610, 13) (729610,)
(20846, 13) (20846,)
Epoch 1/200
22801/22801 [==============================] - 62s 3ms/step - loss: 0.1958 - sparse_categorical_accuracy: 0.9296
Epoch 2/200
22801/22801 [==============================] - 62s 3ms/step - loss: 0.0896 - sparse_categorical_accuracy: 0.9684
Epoch 3/200
22801/22801 [==============================] - 62s 3ms/step - loss: 0.0668 - sparse_categorical_accuracy: 0.9764
Epoch 4/200
22801/22801 [==============================] - 61s 3ms/step - loss: 0.0544 - sparse_categorical_accuracy: 0.9810
Epoch 5/200
22801/22801 [==============================] - 63s 3ms/step - loss: 0.0463 - sparse_categorical_accuracy: 0.9836
Epoch 6/200
22801/22801 [==============================] - 61s 3ms/step - loss: 0.0400 - sparse_categorical_accuracy: 0.9861
Epoch 7/200
22801/22801 [==============================] - 62s 3ms/step - loss: 0.0356 - sparse_categorical_accuracy: 0.9877
Epoch 8/200
22801/22801 [===================

In [0]:
Run('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6
(187614, 13) (187614,)
(20846, 13) (20846,)
Epoch 1/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.3288 - sparse_categorical_accuracy: 0.8825
Epoch 2/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1855 - sparse_categorical_accuracy: 0.9328
Epoch 3/200
5863/5863 [==============================] - 17s 3ms/step - loss: 0.1407 - sparse_categorical_accuracy: 0.9501
Epoch 4/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.1173 - sparse_categorical_accuracy: 0.9585
Epoch 5/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.1035 - sparse_categorical_accuracy: 0.9637
Epoch 6/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.0930 - sparse_categorical_accuracy: 0.9670
Epoch 7/200
5863/5863 [==============================] - 16s 3ms/step - loss: 0.0833 - sparse_categorical_accuracy: 0.9710
Epoch 8/200
5863/5863 [==============================] - 1

In [0]:
Run('data/FNOW/WISDM.npz')

Proposed Model data/FNOW/WISDM.npz
6
(94644, 13) (94644,)
(10516, 13) (10516,)
Epoch 1/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.4187 - sparse_categorical_accuracy: 0.8504
Epoch 2/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.2870 - sparse_categorical_accuracy: 0.8945
Epoch 3/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.2249 - sparse_categorical_accuracy: 0.9173
Epoch 4/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.1841 - sparse_categorical_accuracy: 0.9331
Epoch 5/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.1588 - sparse_categorical_accuracy: 0.9438
Epoch 6/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.1438 - sparse_categorical_accuracy: 0.9500
Epoch 7/200
2958/2958 [==============================] - 8s 3ms/step - loss: 0.1306 - sparse_categorical_accuracy: 0.9545
Epoch 8/200
2958/2958 [==============================] - 8s 3ms/ste

In [0]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Proposed Model data/LOTO/UTD-MHAD2_1s.npz
6
(10233, 25) (10233,)
(1137, 25) (1137,)
Epoch 1/200
320/320 [==============================] - 1s 3ms/step - loss: 0.9448 - sparse_categorical_accuracy: 0.6268
Epoch 2/200
320/320 [==============================] - 1s 3ms/step - loss: 0.5754 - sparse_categorical_accuracy: 0.7648
Epoch 3/200
320/320 [==============================] - 1s 3ms/step - loss: 0.4251 - sparse_categorical_accuracy: 0.8233
Epoch 4/200
320/320 [==============================] - 1s 3ms/step - loss: 0.3654 - sparse_categorical_accuracy: 0.8481
Epoch 5/200
320/320 [==============================] - 1s 3ms/step - loss: 0.3257 - sparse_categorical_accuracy: 0.8690
Epoch 6/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2714 - sparse_categorical_accuracy: 0.8885
Epoch 7/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2447 - sparse_categorical_accuracy: 0.9027
Epoch 8/200
320/320 [==============================] - 1s 3ms/step - loss: 0

In [0]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Proposed Model data/LOSO/UTD-MHAD2_1s.npz
6
(7959, 25) (7959,)
(1137, 25) (1137,)
Epoch 1/200
249/249 [==============================] - 1s 3ms/step - loss: 1.0590 - sparse_categorical_accuracy: 0.5834
Epoch 2/200
249/249 [==============================] - 1s 3ms/step - loss: 0.6479 - sparse_categorical_accuracy: 0.7257
Epoch 3/200
249/249 [==============================] - 1s 3ms/step - loss: 0.5038 - sparse_categorical_accuracy: 0.7907
Epoch 4/200
249/249 [==============================] - 1s 3ms/step - loss: 0.4430 - sparse_categorical_accuracy: 0.8206
Epoch 5/200
249/249 [==============================] - 1s 3ms/step - loss: 0.3785 - sparse_categorical_accuracy: 0.8432
Epoch 6/200
249/249 [==============================] - 1s 3ms/step - loss: 0.3656 - sparse_categorical_accuracy: 0.8553
Epoch 7/200
249/249 [==============================] - 1s 3ms/step - loss: 0.2892 - sparse_categorical_accuracy: 0.8816
Epoch 8/200
249/249 [==============================] - 1s 3ms/step - loss: 0.2

In [0]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Proposed Model data/FNOW/UTD-MHAD2_1s.npz
6
(5544, 25) (5544,)
(616, 25) (616,)
Epoch 1/200
174/174 [==============================] - 0s 3ms/step - loss: 1.0738 - sparse_categorical_accuracy: 0.5722
Epoch 2/200
174/174 [==============================] - 0s 3ms/step - loss: 0.6382 - sparse_categorical_accuracy: 0.7563
Epoch 3/200
174/174 [==============================] - 0s 3ms/step - loss: 0.4422 - sparse_categorical_accuracy: 0.8380
Epoch 4/200
174/174 [==============================] - 0s 3ms/step - loss: 0.3606 - sparse_categorical_accuracy: 0.8656
Epoch 5/200
174/174 [==============================] - 0s 3ms/step - loss: 0.2992 - sparse_categorical_accuracy: 0.8856
Epoch 6/200
174/174 [==============================] - 0s 3ms/step - loss: 0.2231 - sparse_categorical_accuracy: 0.9188
Epoch 7/200
174/174 [==============================] - 0s 3ms/step - loss: 0.2071 - sparse_categorical_accuracy: 0.9239
Epoch 8/200
174/174 [==============================] - 0s 3ms/step - loss: 0.147

In [0]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Proposed Model data/SNOW/UTD-MHAD2_1s.npz
6
(10233, 25) (10233,)
(1137, 25) (1137,)
Epoch 1/200
320/320 [==============================] - 1s 3ms/step - loss: 1.0173 - sparse_categorical_accuracy: 0.6265
Epoch 2/200
320/320 [==============================] - 1s 3ms/step - loss: 0.5414 - sparse_categorical_accuracy: 0.7787
Epoch 3/200
320/320 [==============================] - 1s 3ms/step - loss: 0.4275 - sparse_categorical_accuracy: 0.8247
Epoch 4/200
320/320 [==============================] - 1s 3ms/step - loss: 0.3554 - sparse_categorical_accuracy: 0.8552
Epoch 5/200
320/320 [==============================] - 1s 3ms/step - loss: 0.3164 - sparse_categorical_accuracy: 0.8758
Epoch 6/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2934 - sparse_categorical_accuracy: 0.8841
Epoch 7/200
320/320 [==============================] - 1s 3ms/step - loss: 0.2504 - sparse_categorical_accuracy: 0.9011
Epoch 8/200
320/320 [==============================] - 1s 3ms/step - loss: 0

In [0]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Proposed Model data/LOTO/UTD-MHAD1_1s.npz
21
(33939, 25) (33939,)
(3771, 25) (3771,)
Epoch 1/200
1061/1061 [==============================] - 3s 3ms/step - loss: 2.6910 - sparse_categorical_accuracy: 0.2618
Epoch 2/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.8168 - sparse_categorical_accuracy: 0.4170
Epoch 3/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.5166 - sparse_categorical_accuracy: 0.5025
Epoch 4/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.3190 - sparse_categorical_accuracy: 0.5623
Epoch 5/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.1718 - sparse_categorical_accuracy: 0.6139
Epoch 6/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.0607 - sparse_categorical_accuracy: 0.6485
Epoch 7/200
1061/1061 [==============================] - 3s 3ms/step - loss: 0.9626 - sparse_categorical_accuracy: 0.6795
Epoch 8/200
1061/1061 [==============================] - 3s 3

In [0]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Proposed Model data/LOSO/UTD-MHAD1_1s.npz
21
(26397, 25) (26397,)
(3771, 25) (3771,)
Epoch 1/200
825/825 [==============================] - 2s 3ms/step - loss: 3.1354 - sparse_categorical_accuracy: 0.1779
Epoch 2/200
825/825 [==============================] - 2s 3ms/step - loss: 2.1370 - sparse_categorical_accuracy: 0.3272
Epoch 3/200
825/825 [==============================] - 2s 3ms/step - loss: 1.8075 - sparse_categorical_accuracy: 0.4143
Epoch 4/200
825/825 [==============================] - 2s 3ms/step - loss: 1.5728 - sparse_categorical_accuracy: 0.4808
Epoch 5/200
825/825 [==============================] - 2s 3ms/step - loss: 1.4192 - sparse_categorical_accuracy: 0.5259
Epoch 6/200
825/825 [==============================] - 2s 3ms/step - loss: 1.2756 - sparse_categorical_accuracy: 0.5685
Epoch 7/200
825/825 [==============================] - 2s 3ms/step - loss: 1.1742 - sparse_categorical_accuracy: 0.6065
Epoch 8/200
825/825 [==============================] - 2s 3ms/step - loss: 

In [0]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Proposed Model data/SNOW/UTD-MHAD1_1s.npz
21
(33939, 25) (33939,)
(3771, 25) (3771,)
Epoch 1/200
1061/1061 [==============================] - 3s 3ms/step - loss: 3.0717 - sparse_categorical_accuracy: 0.1724
Epoch 2/200
1061/1061 [==============================] - 3s 3ms/step - loss: 2.1864 - sparse_categorical_accuracy: 0.3096
Epoch 3/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.8032 - sparse_categorical_accuracy: 0.4132
Epoch 4/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.5388 - sparse_categorical_accuracy: 0.4880
Epoch 5/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.3519 - sparse_categorical_accuracy: 0.5483
Epoch 6/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.2137 - sparse_categorical_accuracy: 0.5912
Epoch 7/200
1061/1061 [==============================] - 3s 3ms/step - loss: 1.1007 - sparse_categorical_accuracy: 0.6300
Epoch 8/200
1061/1061 [==============================] - 3s 3

In [0]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Proposed Model data/FNOW/UTD-MHAD1_1s.npz
21
(18432, 25) (18432,)
(2048, 25) (2048,)
Epoch 1/200
576/576 [==============================] - 1s 2ms/step - loss: 3.0288 - sparse_categorical_accuracy: 0.1549
Epoch 2/200
576/576 [==============================] - 1s 2ms/step - loss: 2.3693 - sparse_categorical_accuracy: 0.2639
Epoch 3/200
576/576 [==============================] - 1s 2ms/step - loss: 2.0036 - sparse_categorical_accuracy: 0.3542
Epoch 4/200
576/576 [==============================] - 1s 3ms/step - loss: 1.7106 - sparse_categorical_accuracy: 0.4406
Epoch 5/200
576/576 [==============================] - 1s 2ms/step - loss: 1.5087 - sparse_categorical_accuracy: 0.5049
Epoch 6/200
576/576 [==============================] - 1s 2ms/step - loss: 1.3515 - sparse_categorical_accuracy: 0.5520
Epoch 7/200
576/576 [==============================] - 1s 3ms/step - loss: 1.2068 - sparse_categorical_accuracy: 0.5979
Epoch 8/200
576/576 [==============================] - 1s 2ms/step - loss: 